# Lecture 6: Python for Intrument Control (1.5 Hours) #

### ABSTRACT ###

In this lecture we will see how we can use Python to communicate with scientific instruments. First we will look at two packages that help add unit support to Python. Then we will revisit classes and how they can be usefull for desiging Python interfaces to communicate to devices. We will then work on designing a driver for a demo instrument provided in the [git repo](https://github.com/QuinnPhys/PythonWorkshop-science) for this project.

## Unitful Computing with python-quantities and Pint (15 Minutes)##

Before getting to acutally controling instruments, we'll briefly mention how to use Python to perform numerical computations in a *unitful* way. We'll demonstrate using python-quantities, which provides a *quantity* data type to represent arrays with associated units (and is how units are handled in IntrumentKit). 

For now, let's go on and import python-quantities.

In [1]:
import numpy as np
import quantities as pq

We can now create quantities by specifying both a magnitude and a unit string.

In [2]:
velocity = pq.Quantity(2, 'm / s')
velocity

array(2) * m/s

Units can also be applied by arithmetically multiplying them with an array from numpy or just a python list. The following two ways of adding units are equvalent:

In [3]:
vader = np.array([1,2,3]) * pq.N
luke = [1,2,3] * pq.N
print(vader) 
print(luke)

[ 1.  2.  3.] N
[ 1.  2.  3.] N


Arithmetic operations on quantities respect units.

In [4]:
acceleration = velocity / pq.second
acceleration

array(2.0) * m/s**2

Importantly, operations on quantities must be correct for dimensions, but may use different units.

In [5]:
pq.Quantity(1, 'meter') + pq.Quantity(1, 'foot')

array(1.3048) * m

Now, you may ask what happens when we use dimensionally incorrect units. Look at the below raised error:

In [6]:
pq.Quantity(1, 'meter') + pq.Quantity(1, 'second')

ValueError: Unable to convert between units of "s" and "m"

This looks quite long, but there is basically two main places to look for what the error was: at the top ``ValueError`` and the bottom: ``ValueError: Unable to convert between units of "s" and "m"``. The text in the middle is just tracking where the fuctions were called from and is often not needed to troubleshoot the problem. We can use python statements ``try:`` and ``except`` to help us make errors more readable.

In [7]:
try:
    pq.Quantity(1, 'meter') + pq.Quantity(1, 'second')
except ValueError as ex:
    print(ex)

Unable to convert between units of "s" and "m"


Quanities also support transforming between different unit systems. You can do this in one of two ways, you can directly set the atribute ``.units`` of the object or use the rescale method. 

*NB: if you directly set the atribute by spectifying a named module variable (``pq.hour``) you must use the singular version of the name of the unit (not ``pq.hours``). If you are specifying a unit as a strin

In [8]:
love = pq.Quantity(525600.0, 'minutes')
love.units = pq.hour
print(love)
love.rescale('days')

8760.0 h


array(365.0) * d

Such conversions are also checked for consistent dimensions.

In [9]:
try:
    acceleration.rescale('kg')
except ValueError as ex:
    print(ex)

Unable to convert between units of "m/s**2" and "kg"


Finally, python-quantities also provides a range of useful physical constants that we can use to quickly construct quantities. 

In [10]:
Sz = pq.constants.hbar * np.array([[1, 0], [0, -1]]) / 2
H = pq.constants.gamma_p * pq.Quantity(7, 'tesla') * Sz
print(H)

[[ 3.5  0. ]
 [ 0.  -3.5]] gamma_p*(h/(2*pi))*T


To make it easier to compare values, you can use the ``simplified`` attribute to convert all the units to a simplified form, and can use the ``rescale()`` method to specify particular units.

In [11]:
print(H.simplified)
(H.simplified / (2 * np.pi * pq.constants.hbar)).simplified.rescale('MHz')

[[  9.87424663e-26   0.00000000e+00]
 [  0.00000000e+00  -9.87424663e-26]] kg*m**2/s**2


array([[ 149.02118732,    0.        ],
       [   0.        , -149.02118732]]) * MHz

### Pint units package ###

As we might expect, there is more than one option for a packackage in Python that can help us handle units. [Pint](https://pint.readthedocs.io/en/0.7.2/) is chronologically a newer package, but with a bit more features than quantities. 

First, let's check to see if the package is installed. Try running: 
```bash 
pip show pint
```
in your shell to see if pip knows about the package. If it returns nothing, then you do not have it installed and you should run:
```bash 
pip install pint
```
If if does print out some stuff to the screen, then it will show you where it has the package installed. Let's look at some examples of using the Pint package. 

Pints has this concept of a unit registry, that is a list of all the supported units and their relationship. 

In [12]:
import pint 
ureg = pint.UnitRegistry()

To use these units you simply multiply the magnitude of the value by an instance of a unit object defined by the unit registry. The variable ``time`` then becomes an instance of a quantitiy object:

In [13]:
time = 8.0 * ureg.second
print(time)
type(time)

8.0 second


pint.unit.build_quantity_class.<locals>.Quantity

You can also provide the units as a string that Pint will parse:

In [14]:
time = 8.0 * ureg('seconds')

Being an instance of a quantity type means that we can access atributes of ``time``:

In [15]:
print(time.magnitude)
print(time.units)
print(time.dimensionality)

8.0
second
[time]


We can look at how these ``Quantity`` objects are represented internaly:

In [16]:
print(repr(time))

<Quantity(8.0, 'second')>


This then allows us to be more direct about creating an instance of a ``Quantity``:

In [17]:
distance = ureg.Quantity(3, 'feet')
print(distance)

3 foot


We can perform arithmetic operations with the ``Quantity`` objects with no special syntax:

In [18]:
speed = np.absolute( distance / time )
print(speed)

0.375 foot / second


We can easily convert quantities with the ``.to`` method, which generates a new object with the new units. Else to change the units associated with the object ``speed`` we can ise the ``.ito`` method.

In [19]:
speed.to(ureg.km / ureg.hour)
print(speed)
speed.ito(ureg.km / ureg.hour)
print(speed)

0.375 foot / second
0.41147999999999996 kilometer / hour


Similar to quantities, Pint will warn when a particular unit conversion is not possible.

In [20]:
try:
    speed.to(ureg.N)
except pint.errors.DimensionalityError as ex:
    print(ex)

Cannot convert from 'kilometer / hour' ([length] / [time]) to 'newton' ([length] * [mass] / [time] ** 2)


Lastly, if we want to look up what units are defined in the current registry, we can look at the different unit systems the registry knows about. By default calling ``ureg = UnitRegistry()`` makes units from all systems available.

In [21]:
dir(ureg.sys)

['US', 'cgs', 'imperial', 'mks']

To see what units are defined in a specific unit system just look at the atributes of the system:

In [22]:
dir(ureg.sys.imperial)

['UK_hundredweight',
 'UK_ton',
 'acre_foot',
 'cubic_foot',
 'cubic_inch',
 'cubic_yard',
 'drachm',
 'foot',
 'grain',
 'imperial_barrel',
 'imperial_bushel',
 'imperial_cup',
 'imperial_fluid_drachm',
 'imperial_fluid_ounce',
 'imperial_gallon',
 'imperial_gill',
 'imperial_peck',
 'imperial_pint',
 'imperial_quart',
 'inch',
 'long_hunderweight',
 'long_ton',
 'mile',
 'ounce',
 'pound',
 'quarter',
 'short_hunderdweight',
 'short_ton',
 'square_foot',
 'square_inch',
 'square_mile',
 'square_yard',
 'stone',
 'yard']

## More on Class Inheritance (25 Minutes) ##

Recall that in Python all types are functions which return values of that type. Classes in Python are a way of defining new types. Let's look a little more in-depth at classes and how they can be useful to us for communicating with instruments.

We will use this ``ExampleClass`` to explore our understanding of classes. It inherits the attributes of the ``object`` class which is the most basic class one can inherit from. Unlike the previous examples we are specifing an ``__init__`` definition. This function is called when an instance of the class is created and allows us to specify input arguments. 

In [14]:
class ExampleClass(object):
    '''
    An example of a basic class with attributes and methods.
    '''
    
    def __init__(self, name, value):
        '''
        Initialize the class, set the name, and value for the exponent method.
        '''
        self.name = name
        self.value = value
    
    tally = 0
    
    def test_exponent(self, x):
        self.tally = self.tally + 1
        return x ** self.value
    
    def say_name(self):
        print('My name is {name}.'.format(name = self.name))

You might notice the text inclosed with triple quotes (``"""`` or ``'''``). Triple quotes allow for easily defining a string that includes line breaks, but are otherwise identical to single quotes (``'`` or ``"``). In any case, if the first line of a class or function is a string, then this is called a *docstring* and is basically documentation attached to that class or function. If you try running the below line, you can see that this text is then displayed in the pop-up.

In [16]:
ExampleClass?

We can then create an instance of this class by choosing a name and an exponent value. We can also check on the value of the attribute ``tally``:

In [20]:
inst = ExampleClass('Bob', 3)
inst.say_name()
print(inst.test_exponent(3))
print(inst.tally)

My name is Bob.
27
1


We can check to see if  the atribute ``tally`` is keeping track of how many times we have called the ``test_exponent`` method.

In [61]:
print(inst.test_exponent(4))
print(inst.tally)

64
2


If we want to look to see what attributes that ``ExampleClass`` is inheriting from ``object`` we can use the ``dir`` function to list atributes:

In [63]:
dir(object)

['__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__le__',
 '__lt__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__']

Inheriting these atributes means that they are also valid atributes of ``ExampleClass``. For example, the ``__doc__`` attribute is used by the ``help()`` function and by IPython's and Jupyter's ``?`` commands to provide help. In our case, ``__doc__`` is defined for an instance of our example class ``cls`` and returns the docstring we set.

In [28]:
inst.__sizeof__()

32

We can also make a "subclass" of ``ExampleClass`` that overides one of the methods that we defined in ``ExampleClass``:

In [64]:
class MoreSpecificClass(ExampleClass):
    def say_name(self):
        print('My name is {name}. I raise things to the power of {power}.'.format(name=self.name, power=self.value))

In [65]:
better_inst = MoreSpecificClass("Alice", 2)
better_inst.say_name()

My name is Alice. I raise things to the power of 2.


Having this inheriting structure for classes makes it easy to share definitions for methods, which can reduce copy/pasting of code and make things more readable. Critically, class inheritance also allows for redefining behavior without having to change how a class is used. In the case of instruments, a base class might represent a particular kind of instrument, such that code using subclasses of that class does not need to know which subclass it is using. We can write code for an entire set of different instruments by abstracting away the *interface* that our code uses to talk to each instrument. This interface is sometimes known as an *application-programming interface*, or *API*, as was [made famous recently](https://parkerhiggins.net/wp-content/uploads/2016/05/reimplementanapi.png). 

Python lets us represent this process of abstraction with *abstract classes* (sometimes called *abstract base classes*, in an effort to arrive at *ABCs* as an initialism). Unlike the classes we've seen so far, an abstract class is incomplete in the sense that you can't actually make new instances, but instead must make *concrete* subclasses which fill in the missing parts. Thus, an abstract class lets us write the demand that subclasses fill in and completely specify the behavior for a given interface before they can be considered concrete.

Let's look at a quick example. In Python, support for ABCs is provided by the ``abc`` package. A class that can be made abstract must have ``abc.ABCMeta`` as its *metaclass*. While metaclasses are fascinating and wonderfully useful, they are also beyond the scope of this workshop, so for now, we'll note that a metaclass lets us add new concepts to the idea of a class itself, rather than to instances of it. Thus, ``ABCMeta`` adds the concept of abstraction to classes. We use the ``future`` package to specify metaclasses, since the syntax for this is different in Python 2 or 3. With that aside, then, our wonderfully compelling and not at all contrived example follows.

In [28]:
import abc
from future.utils import with_metaclass

In [30]:
class AbstractFoo(with_metaclass(abc.ABCMeta, object)):
    @abc.abstractmethod
    def bar(self):
        """
        bar should really do something.
        """
        pass

We'll get back to what ``@abc.abstractmethod`` is doing in a moment, but for now let's take it for granted that it marks a method as being abstract. Because ``AbstractFoo`` has at least one abstract method, it is an abstract class, and we cannot make new instances yet.

In [36]:
try:
    foo = AbstractFoo()
except TypeError as ex:
    print(ex)

Can't instantiate abstract class AbstractFoo with abstract methods bar


If we give ``AbstractFoo`` a specification of ``bar()`` by using inheritance, then we're in the clear.

In [37]:
class Foo(AbstractFoo):
    def bar(self):
        print("<3")

In [38]:
foo = Foo()
foo.bar()

<3


Below, we'll explore this idea further now by looking at some abstract classes that could describe kinds of instruments we might want to control with Python.

### A Brief Aside About Decorators ###

*NB: This part's a bit more advanced. Please don't worry if it doesn't all make sense the first time through, or even the second. It's not that hard in some ways, but it's also a bit different and that takes a lot of getting used to.*

First, though, what was ``abc.abstractmethod`` doing above? Notice that the line started with an at-character (``@``). In Python, this denotes that a function is being used as a *decorator*, a fancy name for a function that takes a function and returns a function. Let's look at a very simple case, in which the decorator does nothing (is the identity decorator), but has a side effect of printing something.

In [41]:
def loud_identity(fn):
    print("Defining {}...".format(fn))
    return fn

@loud_identity
def bar(x):
    return x ** 2

print(bar(2))
print(bar(3))

Defining <function bar at 0x0000000007CC0208>...
4
9


We can make more complicated decorators, too. For instance, we could make a decorator to represent applying a function twice, similar to what we did in [Lecture 2](https://nbviewer.jupyter.org/github/QuinnPhys/PythonWorkshop-science/blob/master/lecture-2-python-general.ipynb) to explore that functions are values.

In [43]:
from functools import wraps

def apply_twice(fn):
    @wraps(fn)
    def new_fn(x):
        return fn(fn(x))
    return new_fn

@apply_twice
def bar(x):
    return x * 2 + "!"

In [44]:
bar("ab")

'abab!abab!!'

Here, the ``wraps`` decorator is provided by the Python standard library to copy docstrings, function names and other metadata to a new function; it's a decorator that helps us to write decorators.

*NB: Note that ``wraps`` takes an argument. This is an example of what's called [currying](https://en.wikipedia.org/wiki/Currying), in that ``wraps`` takes a function and returns a decorator. That is, a function from functions to functions from functions to functions.*

Back to ``abc.abstractmethod``, then, what does it do?

In [48]:
AbstractFoo.bar.__isabstractmethod__

True

In [49]:
AbstractFoo.__abstractmethods__

frozenset({'bar'})

That's the crux of it, really. The ``abstractmethod`` decorator sets a new attribute on the method (``bar.__isabstractmethod__``) and adds the method to a special attribute on the class (``AbstractFoo.__abstractmethods__``). Python then uses these internally (hence the double-underscore names) to track that ``bar`` is abstract. In fact, if we take the wholly inadvisable step of modifying ``__abstractmethods__``, we can even trick Python into letting us make instances of abstract classes!

*NB: Don't do this. Seriously.*

In [55]:
AbstractFoo.__abstractmethods__ = {}
AbstractFoo()

## Let's build up an Instrument and talk to it! (50 Minutes) ##

To actualy put what we have learned now about abstract classes to use, we will use the ``InstrumentKit`` Python package to write a class that can talk to a fake instrument. This instrument is also writen in Python with a package that allows you to make GUIs ([PyQt4](http://pyqt.sourceforge.net/Docs/PyQt4/introduction.html#pyqt4-components)).
It will not matter though that it is also writen in Python as we will *only* interface with it via text strings, which is the basis for many instrument communication protocols. 

To install our "instrument" simply, open a shell and navigate to where you checked out the git repo for this workshop. Once there navigate to the ``epqis16-demos`` folder. In this folder, we have a package that you can install to python by running the following command:

```bash
$ python setup.py install
```

Now to actually run the instrument simply run this command in the shell you have open:

```bash
$ epqis16 demo_instrument
```

The shell should now show  ``Waiting for a connection from InstrumentKit on port 8042.``. We will leave it like this for now.

### The problem ###

Consider the following situation: your experiment needs constant babysitting to set values and make sure that it functions correctly. You decide that to make your life simpler you will write a program that can set all the measurements needed and save the output data. The device you need to control is a four channel DC power supply. Opening the manual you find the following table for what commands you can send to the device:

| SPCI command              | Example usage                                                      | Options                                                                                 | Description                                           |
|---------------------------|--------------------------------------------------------------------|-----------------------------------------------------------------------------------------|-------------------------------------------------------|
| ``*IDN?``                | Query return:<br>``EPQIS16 Demonstration Instrument``              |                                                                                         | Find out what the name of the instument is.           |
| ``CH<ch> VOLTS?``          | ``CH1 VOLTS?``<br>``CH2 VOLTS?``<br>Query return:<br>``{value in mV}`` | ``<ch> = {1, 2, 3, 4}``                                                      | Check what the ouput voltage is set to  (returns mV). |
| ``CH<ch> VOLTS <val>``    | ``CH1 VOLTS 314``<br>``CH3 VOLTS 159``                               | ``<ch> = {1, 2, 3, 4}``<br>``<val> = voltage value in mV`` | Set the voltage output (provided value is mV).        |
| ``CH<ch> ENABLE <state>``  | ``CH2 ENABLE OFF``<br>``CH4 ENABLE ON``                              | ``<ch> = {1, 2, 3, 4}``<br>``<state> = {ON, OFF}``<br>Default: ``OFF``        | Toggle the voltage channel output on/off.             |     
| ``CH<ch> ENABLE?``         | ``CH1 ENABLE?``<br>``CH2 ENABLE?``<br> Query return:<br>``{ON, OFF}``  | ``<ch> = {1, 2, 3, 4}``                                                      | Check the ouput status of a voltage channel.          | 

We can do some inital tests to see if we can send or recieve commands from the instrument. Since we saw (intentionally erronious as you will find in real life) that the manual specified the communication protocol as SPCI. Asuming that is was true, we could consider treating our instrument as a generic SCPI intrument from the ``InstrumentKit`` package. Let's start by importing as ik:

In [2]:
import instruments as ik

Now, since we read elsewhere in the manual for this device that to connect to it we use a tcpip protocol and connect to port 8042. All devices will have some section descibing how to do this initial connection, but because we are using a demo "instrument" anyway, this is how we have chosen to connect. 

The idea now is that we could use the communication protocols that IK has built in to see if we can find and connect to the device. We start by creating an instance of a generic instrument class that only knows how to communicate via SCPI. 

In [4]:
ins = ik.generic_scpi.SCPIInstrument.open_tcpip("localhost", 8042)
type(ins)

instruments.generic_scpi.scpi_instrument.SCPIInstrument

*NB: The ``open_tcpip`` method above works on classes, not instances of classes, and is hence known as a class method. This is useful to us here, as we don't yet have an instance of a class for ``open_tcpip`` to act on. Rather, it provides a way of creating class instances in the first place.*

Having done this, a little pop-up window should have appeared on your screen. This is the front panel of our demo device, a 4 channel DC power supply! Now, let's see if we can send/recieve sensable things to/from the device using the ``query`` and ``sendcmd`` methods of the SCPIInstrument class. *NB: you could get similar sendcmd/query commands from other python serial communication packages, but we will look at those in IK since we will also use IK for other things.*

In [5]:
ins.query("*IDN?")

'EPQIS16 Demonstration Instrument'

In [6]:
ins.query("CH2 VOLTS?")

'0.0'

In [7]:
ins.sendcmd("CH2 VOLTS 42")
ins.query("CH2 VOLTS?")

'42.0'

In [8]:
ins.query("CH2 ENABLE?")

'OFF'

In [9]:
ins.sendcmd("CH1 ENABLE ON")

This is cool! Go ahead and try turning the channels on and off in the GUI and see if that is correctly reported by the instrument. At this point, we have a decision point. We could write all of our code by just constructing the strings that are the commands we want to send to the device. Or we could try to something more *generalized*; in that we abstract the communication details specific to this device to a seperate file and just have a generalized interface for other code to use the device. The advantanges to the latter approach mean that if for some reason you need to switch to another device, you can just swap out the specific driver file, or perhaps you need to communicate to another similar instrument and don't want to risk having copies of the same code floating around. 

In principal the startup costs for both of these approaches are the same, in that you have to setup code to do the "translation" to the intstrument. However, as we suggest here there is a lot to be gained by using a package such as IK (others are listed at the end of the lecture) to set up abstract instrument classes as well as classes that handle many types of common communication interfaces (GPIB, USB, VISA, Serial, VXI11, etc). All of these templates set up in these packages you could also write yourself, but the question is: "wouldn't you rather be doing your experiment and not coding?" Suffice it to say we will now look at how we could use abstract classes from IK to write a driver for our demo instrument.   

### The Solution ###

Now as we have seen with class inheritance, IntrumentKit has a whole host of what are called abstract instruments that set up templates for what attributes and methods would be good to have in a class for a particular kind of instrument. We can take a look at what "templates" InstrumentKit has setup by using the ``help`` function:

In [81]:
help(ik.abstract_instruments)

Help on package instruments.abstract_instruments in instruments:

NAME
    instruments.abstract_instruments - Module containing instrument abstract base classes and communication layers

PACKAGE CONTENTS
    comm (package)
    electrometer
    function_generator
    instrument
    multimeter
    oscilloscope
    power_supply
    signal_generator (package)

DATA
    absolute_import = _Feature((2, 5, 0, 'alpha', 1), (3, 0, 0, 'alpha', 0...

FILE
    /home/skaiser/anaconda3/lib/python3.5/site-packages/instruments/abstract_instruments/__init__.py




We can see that there is an abstract instrument called ``power_supply``, so since we are trying to write a driver for one let's look at that in more detail:

In [30]:
help(ik.abstract_instruments.PowerSupply)

Help on class PowerSupply in module instruments.abstract_instruments.power_supply:

class PowerSupply(instruments.abstract_instruments.instrument.Instrument)
 |  Abstract base class for power supply instruments.
 |  
 |  All applicable concrete instruments should inherit from this ABC to
 |  provide a consistent interface to the user.
 |  
 |  Method resolution order:
 |      PowerSupply
 |      instruments.abstract_instruments.instrument.Instrument
 |      __builtin__.object
 |  
 |  Data descriptors defined here:
 |  
 |  channel
 |      Gets a channel object for the power supply. This should use
 |      `~instruments.util_fns.ProxyList` to achieve this.
 |      
 |      This is an abstract method.
 |      
 |      :rtype: `PowerSupplyChannel`
 |  
 |  current
 |      Gets/sets the output current for all channel on the power supply.
 |      This is an abstract method.
 |      
 |      :type: `~quantities.quantity.Quantity`
 |  
 |  voltage
 |      Gets/sets the output voltage for all

Now this is a lot of information, but of interest here is the lines starting at ``class PowerSupply``:

```
class PowerSupply(instruments.abstract_instruments.instrument.Instrument)
     |  Abstract base class for power supply instruments.
     |  
     |  All applicable concrete instruments should inherit from this ABC to
     |  provide a consistent interface to the user.
     |  
     |  Method resolution order:
     |      PowerSupply
     |      instruments.abstract_instruments.instrument.Instrument
     |      builtins.object
     |  
     |  Data descriptors defined here:
     |  
     |  channel
     |      Gets a channel object for the power supply. This should use
     |      `~instruments.util_fns.ProxyList` to achieve this.
     |      
     |      This is an abstract method.
     |      
     |      :rtype: `PowerSupplyChannel`
     |  
     |  current
     |      Gets/sets the output current for all channel on the power supply.
     |      This is an abstract method.
     |      
     |      :type: `~quantities.quantity.Quantity`
     |  
     |  voltage
     |      Gets/sets the output voltage for all channel on the power supply.
     |      This is an abstract method.
     |      
     |      :type: `~quantities.quantity.Quantity`
     ```

This tells us that the class ``PowerSupply`` has three abstract methods, namely ``channel``, ``current``, and ``voltage``. As we discussed previously, if we want to make a concrete class that inherits from ``PowerSupply`` then we have to override these methods and give them a definition before we can actually make an instance of our concrete class.

The class ``PowerSupplyChannel`` is mentioned above as the return type for the channel method. Let's look at what the abstract class for a channel of a power supply looks like:

In [11]:
help(ik.abstract_instruments.PowerSupplyChannel)

Help on class PowerSupplyChannel in module instruments.abstract_instruments.power_supply:

class PowerSupplyChannel(__builtin__.object)
 |  Abstract base class for power supply output channels.
 |  
 |  All applicable concrete instruments should inherit from this ABC to
 |  provide a consistent interface to the user.
 |  
 |  Data descriptors defined here:
 |  
 |  __dict__
 |      dictionary for instance variables (if defined)
 |  
 |  __weakref__
 |      list of weak references to the object (if defined)
 |  
 |  current
 |      Gets/sets the output current for the power supply channel. This is an
 |      abstract method.
 |      
 |      :type: `~quantities.quantity.Quantity`
 |  
 |  mode
 |      Gets/sets the output mode for the power supply channel. This is an
 |      abstract method.
 |      
 |      :type: `~enum.Enum`
 |  
 |  output
 |      Gets/sets the output status for the power supply channel. This is an
 |      abstract method.
 |      
 |      :type: `bool`
 |  
 |  vol

As an exercise for now, take some time to look at what some concrete classes for instruments look like [here](https://github.com/Galvant/InstrumentKit/tree/master/instruments/). Notably, look at what abstract classes the real instrument classes are inherting from and what methods they had provide definitions for because of the abstract class. 

After looking at some of these concrete instrument classes, have a shot at writing your own for our demo instrument! A starting hint is that the class definition should probably start by inheriting from the PowerSupply class. Don't forget to override the required methods!

In [60]:
class EpqisDemoInstrument(ik.abstract_instruments.PowerSupply):
    class Channel(ik.abstract_instruments.PowerSupplyChannel):
        pass

You can test your new instrument by using the ``open_test`` class method, which specifies that the "instrument" is actually whatever you type into Jupyter Notebook.

In [69]:
ins = EpqisDemoInstrument.open_test()

In [68]:
ins.query("*IDN?")

 <- '*IDN?\n' 
Desired Response: Foobar


u'Foobar'

If you want to test with the real fake instrument, use the ``open_tcpip`` class method instead:

In [ ]:
ins = EpqisDemoInstrument.open_tcpip('localhost', 8042)

Let's take a look at a possible way you could write this driver file for the demo instrument. This example driver file was a part of the package you imported earlier, so let's import it explicitly now:

In [32]:
from epqis16_demos import EpqisDemoInstrument

Just like with the base instrument class, we start by making an instance of our new instrument class called ``EpqisDemoInstrument``. Since ``PowerSupply`` inherits methods from the ``Instrument`` class, and since that class has the definition for the ``open_tcpip`` method we used before, we can use it again to connect to the instrument.

In [34]:
ik_ins = EpqisDemoInstrument.open_tcpip("localhost", 8042)

Now comes the fun part, just like we have done all along with our trivial example classes, we can just ask for the voltage atribute of a particular channel on our device:

In [35]:
ik_ins.channel[1].voltage

array(0.0) * V

Similarly, we can set the value of attributes of the channels of the devices, and e:

In [36]:
ik_ins.channel[2].output = True

In [37]:
ik_ins.voltage = 12
ik_ins.channel[1].voltage = pq.Quantity(314, 'mV')
ik_ins.channel[1].voltage

array(0.314) * V

In [82]:
ik_ins.current

NotImplementedError: This instrument does not support querying or setting the output current.

Let's take a look at the source for this driver file (with some comments removed to shorten it), and look at how the required ``channel``, ``voltage``, and ``current`` methods are set for the ``EpqisDemoInstrument`` class.

```python
#!/usr/bin/env python
# -*- coding: utf-8 -*-
"""
Provides support for the epqis16 Demonstration Instrument.
"""

# IMPORTS #####################################################################

from __future__ import absolute_import
from __future__ import division

import quantities as pq

from instruments.abstract_instruments import (
    PowerSupply,
    PowerSupplyChannel,
)

from instruments.util_fns import assume_units, ProxyList, bool_property
from instruments.generic_scpi import SCPIInstrument

# CLASSES #####################################################################

class EpqisDemoInstrument(PowerSupply, SCPIInstrument):

    _channel_count = 4

    def __init__(self, filelike):
        super(EpqisDemoInstrument, self).__init__(filelike)

    # PROPERTIES #
    @property
    def channel(self):
        return ProxyList(self, EpqisDemoInstrument.Channel, range(self._channel_count))

    @property
    def voltage(self):
        return [
            self.channel[i].voltage for i in range(self._channel_count)
        ]

    @voltage.setter
    def voltage(self, newval):
        if isinstance(newval, (list, tuple)):
            if len(newval) is not self._channel_count:
                raise ValueError('When specifying the voltage for all channels '
                                 'as a list or tuple, it must be of '
                                 'length {}.'.format(self._channel_count))
            for channel, new_voltage in zip(self.channel, newval):
                channel.voltage = new_voltage
        else:
            for channel in self.channel:
                channel.voltage = newval

    @property
    def current(self):
        raise NotImplementedError('This instrument does not support querying '
                                  'or setting the output current.')

    @current.setter
    def current(self, newval):
        raise NotImplementedError('This instrument does not support querying '
                                  'or setting the output current.')
```

We can see that the device is defined to have 4 channels with ``_channel_count``. The @property and @____.setter lines are called decorators and basically allow a function to "wrap" the one that is defined right below it. That means that the decorator function is called first with the argument of the function that it is wrapping. For example then for the function voltage(self), it is actually called as property(voltage)(self).

Also defined in the ``EpqisDemoInstrument`` is what a channel for this

```python
 class Channel(PowerSupplyChannel):

        def __init__(self, parent, idx):
            self._parent = parent
            self._idx = idx + 1

        # COMMUNICATION METHODS #

        def sendcmd(self, cmd):
            self._parent.sendcmd("CH{} {}".format(self._idx, cmd))

        def query(self, cmd):
            return self._parent.query("CH{} {}".format(self._idx, cmd))

        # PROPERTIES #
        @property
        def mode(self):
            raise NotImplementedError('This instrument does not support querying '
                                      'or setting the output current.')

        @mode.setter
        def mode(self, newval):
            raise NotImplementedError('This instrument does not support querying '
                                      'or setting the output current.')

        @property
        def voltage(self):
            value = self.query("VOLTS?")
            return assume_units(float(value), pq.millivolt).rescale(pq.volt)

        @voltage.setter
        def voltage(self, newval):
            newval = assume_units(newval, pq.volt).rescale(pq.millivolt).magnitude
            self.sendcmd("VOLTS {}".format(newval))

        @property
        def current(self):
            raise NotImplementedError('This instrument does not support querying '
                                      'or setting the output current.')

        @current.setter
        def current(self, newval):
            raise NotImplementedError('This instrument does not support querying '
                                      'or setting the output current.')

        output = bool_property(
            "ENABLE",
            inst_true="ON",
            inst_false="OFF",
            doc="""
            Sets the outputting status of the specified channel.

            This is a toggle setting that can be ON or OFF. 

            :type: `bool`
            """
        )

    # PROPERTIES #
    @property
    def channel(self):
        return ProxyList(self, EpqisDemoInstrument.Channel, range(self._channel_count))

    @property
    def voltage(self):
        return [
            self.channel[i].voltage for i in range(self._channel_count)
        ]

    @voltage.setter
    def voltage(self, newval):
        if isinstance(newval, (list, tuple)):
            if len(newval) is not self._channel_count:
                raise ValueError('When specifying the voltage for all channels '
                                 'as a list or tuple, it must be of '
                                 'length {}.'.format(self._channel_count))
            for channel, new_voltage in zip(self.channel, newval):
                channel.voltage = new_voltage
        else:
            for channel in self.channel:
                channel.voltage = newval

    @property
    def current(self):
        raise NotImplementedError('This instrument does not support querying '
                                  'or setting the output current.')

    @current.setter
    def current(self, newval):
        raise NotImplementedError('This instrument does not support querying '
                                  'or setting the output current.')
```

## Lots of options of starting points, don't reinvent the wheel! ## 

We all have no more valuable resource than our own time, so there is little point to reimplimenting functionality that exsists in other packages that will work in your Python workflow. As ususal, google your instrument to see if there is a Python package that already supports it (or if the manufacuter can supply control code samples!). Here are some popular packages that together cover a lot of instruments. 

- [InstrumentKit](http://instrumentkit.readthedocs.io/en/latest/index.html)
- [Instrumental](http://instrumental-lib.readthedocs.io/en/latest/)
- [QuDi](http://qosvn.physik.uni-ulm.de/trac/qudi)  
- [Python IVI](http://alexforencich.com/wiki/en/python-ivi/readme)